In [1]:
import pandas as pd

df = pd.read_excel (r'C:\Users\thomas\Documents\università\valetti\line.xlsx') 
df.columns = ['s', 'v_norm', 'v_inib', 'conc_inib'] 
df
  #bisogna fare in modo che le colonne si rinomino da sole in s, v_norm, v_inib, in modo che si adatti allo script
    # la griglia deve essere in alto a sinistra nel foglio excel, e avere una sola riga di nomi, e in ordine di colonna: substr, v normale, v inibita

,s,v_norm,v_inib,conc_inib
0,0.033,0.01008,0.00657,5.0
1,0.050,0.01215,0.00837,NaN
2,0.066,0.01341,0.00981,NaN
3,0.100,0.01521,0.01179,NaN
4,0.200,0.01728,0.01476,NaN


In [2]:
from bokeh.plotting import figure, show, output_notebook
param = dict(width = 700, height = 250,
             tools = 'wheel_zoom, reset,pan, box_zoom',
             tooltips = [( 'substrate',   '@s'), ( 'velocity',   '@v')],
            )
output_notebook()

Loading BokehJS ...

In [3]:
p = figure(x_axis_label='Concentrazione substrato',
           y_axis_label='Velocità di reazione',
           **param,
           )
p.circle( 's', 'v_norm', source=df )
p.square('s', 'v_inib', source=df) 
show( p )

In [4]:
df['1/s'] = 1/df['s']
df['1/v_norm'] = 1/df['v_norm']
df['1/v_inib']= 1/df['v_inib']
df


,s,v_norm,v_inib,conc_inib,1/s,1/v_norm,1/v_inib
0,0.033,0.01008,0.00657,5.0,30.303030,99.206349,152.207002
1,0.050,0.01215,0.00837,NaN,20.000000,82.304527,119.474313
2,0.066,0.01341,0.00981,NaN,15.151515,74.571216,101.936799
3,0.100,0.01521,0.01179,NaN,10.000000,65.746220,84.817642
4,0.200,0.01728,0.01476,NaN,5.000000,57.870370,67.750678


In [5]:
from scipy.stats import linregress
slope_norm, intercept_norm, r_value, p_value, std_err = linregress( df['1/s'], df['1/v_norm'] )
slope_norm, intercept_norm   
'The slope of the non-inhibited enzyme is {}, and the intercept of the non-inibited enzyme is {}'.format(slope_norm, intercept_norm)

'The slope of the non-inhibited enzyme is 1.6373268490967405, and the intercept of the non-inibited enzyme is 49.59365880485513'

In [6]:
from scipy.stats import linregress
slope_inib, intercept_inib, r_value, p_value, std_err = linregress( df['1/s'], df['1/v_inib'] )
'The slope of the inhibited enzyme is {}, and the intercept of the inibited enzyme is {}'.format(slope_inib, intercept_inib)  

'The slope of the inhibited enzyme is 3.3480784248271647, and the intercept of the inibited enzyme is 51.36366109799226'

In [7]:
from numpy import array 

In [8]:
#faccio un grafico con le 2 rette per capire il tipo di inibizione
p = figure(title = 'Linearizzazione di Lineweaver-Burk',
            x_axis_label= '1/s', 
            y_axis_label= '1/v',
            **param,
            )
p.circle( '1/s', '1/v_norm', source=df )
xmax = df['1/s'].max() 
x = array( [-3, xmax] )       
y = x * slope_norm + intercept_norm 
p.line( x, y, color='green')

p.square( '1/s', '1/v_inib', source=df)
     
y = x * slope_inib + intercept_inib 
p.line( x, y, color='red')
show( p )

In [9]:
#calcolo il punto di incontro delle rette, fondamentale per definire il tipo di inibizione
slope_x= slope_norm-slope_inib

intercept_x= -intercept_norm+intercept_inib
intersec_x=intercept_x/slope_x
intersec_x
intersec_y= slope_norm*intersec_x+intercept_norm
intersec_y

p.triangle( intersec_x, intersec_y, color='black')
show( p )
print("The intersection of the 2 lines is at:(" +str(intersec_x)+ ','+str(intersec_y)+')')

The intersection of the 2 lines is at:(-1.034634319937074,47.899624253825216)


In [10]:
#si studia ora l'inibizione con la prima linearizzazione 
#1/v = 1/s * B *(Km/Vmax) + A/Vmax


In [11]:
a=slope_inib/slope_norm
b=intercept_inib/intercept_norm
Kei= df['conc_inib']/(a-1)
print("The facteor a is: "+str( a)+', and the factor b is: '+str( b)+' and the Kei is: '+str(Kei)+' .')
# tutti i NaN sono dovuti alla mancanza del valore dell inibente, per ora non so come toglierlo

The facteor a is: 2.044844269593569, and the factor b is: 1.0356900929633335 and the Kei is: 0    4.785402
1         NaN
2         NaN
3         NaN
4         NaN
Name: conc_inib, dtype: float64 .
